<a href="https://colab.research.google.com/github/yunmokoo/SPY_RNN/blob/main/SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install yfinance
import requests
import json

In [ ]:
import yfinance as yf

symbol = "SPY"
data = yf.download(symbol, start = "2022-01-01", end = "2023-06-11")

print(data)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-01-03  476.299988  477.850006  473.850006  477.709991  466.563354   
2022-01-04  479.220001  479.980011  475.579987  477.549988  466.407043   
2022-01-05  477.160004  477.980011  468.279999  468.380005  457.451050   
2022-01-06  467.890015  470.820007  465.429993  467.940002  457.021271   
2022-01-07  467.950012  469.200012  464.649994  466.089996  455.214478   
...                ...         ...         ...         ...         ...   
2023-06-05  428.279999  429.619995  426.369995  427.100006  425.519379   
2023-06-06  426.670013  428.579987  425.989990  428.029999  426.445923   
2023-06-07  428.440002  429.619995  426.109985  426.549988  424.971375   
2023-06-08  426.619995  429.600006  425.820007  429.130005  427.541870   
2023-06-09  429.959991  431.989990  428.869